In [1]:
# fasttext model
# 参考了https://github.com/brightmart/text_classification
# 因为fasttext在windows上的是在是安装不了，所以就看了df群里一个大佬的源码实现
# 顺便自己也理解一下这个模型的思路，其实还是挺简单的

In [1]:
import tensorflow as tf
import numpy as np

class fasttext:
    """
    因为我安装不上fasttext，所以就参考了git上别人实现的源码，自己写一遍吧。
    我找不到facebook的文献，所以只能看git。但是git上又有很多不同版本的代码，里面的思想好像差了有点多。
    一些用nce_loss，一些用cross_entropy loss，我搞不太清楚。因为nce_loss是做embed里用的，cross_entropy loss是做分类的。
    但是根据我以前的经验，我决定用的是cross_entropy loss。
    但不管怎么说，fasttext是个很简单的模型：把ngram向量化，做个embed取平均然后做个softmax就行了。
    这个类不负责数据预处理，只做训练模型。
    """
    def __init__(self,label_size,learning_rate,batch_size,decay_step,decay_rate,num_sampled,vocab_size,embed_size,istraining,sentence_len):
        """初始化参数"""
        self.label_size = label_size
        self.batch_size = batch_size
        self.num_sampled = num_sampled
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.istraing = istraining
        self.learning_rate = learning_rate
        self.sentence_len = sentence_len

        # 初始化placeholder
        self.sentence = tf.placeholder(tf.int32,[None,None],name='sentence')
        print(self.sentence.shape)
        self.labels = tf.placeholder(tf.int32,[None],name="labels")

        self.global_step = tf.Variable(0,trainable=False,name='global_step')
        self.epoch_step = tf.Variable(0,trainable=False,name='epoch_step')
        self.epoch_increment = tf.assign(self.epoch_step,tf.add(self.epoch_step,tf.constant(1)))
        self.decat_step,self.decay_rate = decay_step,decay_rate

        #再次初始化，不然assign会加1
        self.epoch_step = tf.Variable(0, trainable=False, name='epoch_step')
        self.instantiate_weight()
        self.logits = self.inference()
        if not istraining:
            return
        self.loss_val = self.loss()
        self.train_op = self.train()
        self.prediction = tf.argmax(self.logits,axis=1,name="prediction")
        correct_prediction = tf.equal(tf.cast(self.prediction,tf.int32),self.labels)
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="Accuracy")


    def instantiate_weight(self):
        # 还不太懂
        self.Embedding = tf.get_variable('embedding',[self.vocab_size,self.embed_size])
        self.W = tf.get_variable('W',[self.embed_size,self.label_size])
        self.b = tf.get_variable('b',[self.label_size])

    def inference(self):
        # 一样不懂
        sentence_embed = tf.nn.embedding_lookup(self.Embedding,self.sentence)
        self.sentence_embed = tf.reduce_mean(sentence_embed,axis=1)
        logit = tf.matmul(self.sentence_embed,self.W)+self.b
        return logit

    def loss(self):
        if self.istraing:
            labels = tf.reshape(self.labels, [-1])
            labels = tf.expand_dims(labels, 1)
            labels_one_hot = tf.one_hot(self.labels,self.label_size)
            # loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_one_hot,logits=self.logits)
            # print(loss.shape)
            loss = tf.reduce_mean(tf.nn.nce_loss(weights=tf.transpose(self.W),
                                  biases=self.b,
                                  labels=labels,
                                  inputs=self.sentence_embed,
                                  num_sampled=self.num_sampled,
                                  num_classes=self.label_size,partition_strategy='div'
                                  ))
        else:
            labels_one_hot = tf.one_hot(self.labels,self.label_size)
            loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_one_hot,logits=self.logits)
            print('loss0:',loss)
            loss = tf.reduce_mean(loss,axis=1)
            print("loss1:",loss)
        #需要了解
        l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name])
        return loss

    def train(self):
        #需要了解了解
        learning_rate = tf.train.exponential_decay(self.learning_rate,self.global_step,self.decat_step,self.decay_rate,staircase=True)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val,global_step = self.global_step,learning_rate=learning_rate,optimizer="Adam")
        return train_op

In [2]:
import pandas as pd
data = pd.read_csv('C:/Users/Administrator/Desktop/'+'train.csv')
from nltk import word_tokenize
data['word_split'] = data.text.apply(lambda x:word_tokenize(str(x)))
wordlist = []
for i in data.word_split:
    for x in i:
        if x not in wordlist:
            wordlist.append(x)
vocab_size = len(wordlist)
word2int = {}
int2word = {}
for i,i_index in enumerate(wordlist):
    word2int[i]=i_index
    int2word[i_index] = i
data['sentence_len'] = data.word_split.apply(lambda x:len(x))

In [3]:
data['word_index'] = data.word_split.apply(lambda x:[int2word[i] for i in x])

In [4]:
data['label'] = data.author.apply(lambda x:0 if x=='EAP' else 1 if x=='HPL' else 2)

In [5]:
num_classes=3
learning_rate=0.01
batch_size=1
decay_steps=1000
decay_rate=0.9
sequence_length=5
vocab_size=len(word2int)
embed_size=100
is_training=True

In [6]:
import numpy as np
np.array(data.word_index[0])

array([ 0,  1,  2,  3,  2,  4,  5,  6,  7,  8,  9, 10, 11,  8, 12, 13, 14,
       15, 16, 17, 18, 19, 20,  2, 21, 22, 23, 10, 24, 25, 16, 26, 27,  2,
       28, 29, 30,  8, 10, 31, 14, 32, 33, 34, 35, 10, 36, 37])

In [10]:
fastText = fasttext(num_classes, learning_rate, batch_size, decay_steps, decay_rate,2,vocab_size,embed_size,is_training,5)
with tf.Session() as sess:
    for i in range(19579):
        sess.run(tf.global_variables_initializer())
        input_x = np.array([data.word_index[i]],dtype=np.int32)
        input_y = input_y = np.array([data.label[i]], dtype=np.int32)
        for i in range(10):
            loss,acc,predict,_=sess.run([fastText.loss_val,fastText.accuracy,fastText.prediction,fastText.train_op],
                                        feed_dict={
                                            fastText.labels:input_y,
                                            fastText.sentence:input_x})
            print("loss:",loss,"acc:",acc,"label:",input_y,"prediction:",predict)

(?, ?)


ValueError: Variable embedding already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2630, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)
